# Ideal Binary Mask

The idea is to retain parts of a mixture where the target sound is stronger than the acoustic background, and discard the rest

The definition of the ideal binary mask (IBM)



In [61]:
import sys
sys.path.append("..")

import librosa
import numpy as np
import scipy
from IPython.display import Audio

from process import get_stft

In [62]:
clean_audio_dir = "../data/raw/edinburgh-noisy-speech-db/clean_trainset_28spk_wav/"
log_trainset = "../data/raw/edinburgh-noisy-speech-db/log_trainset_28spk.txt"

def load_files():
    audio_files = []
    # list files
    f = open(log_trainset, "r")
    for x in f:
        audio_files.append(x.split()[0] + ".wav")
    f.close()

    return audio_files

audio_files = load_files()
audio_files = [clean_audio_dir + x  for x in audio_files]
audio_files[0]

y, sr = librosa.load(audio_files[0])
Audio(y,rate=sr)

In [63]:
n = 0.15*np.random.uniform()*np.amax(y) * np.random.normal(size=y.shape[0])

In [74]:
window_length = 256
overlap = 0.75
sampling_rate = 22050
num_segments = 8

S_abs = np.abs(get_stft(y, sr, window_length, overlap, sampling_rate))
N_abs = np.abs(get_stft(n, sr, window_length, overlap, sampling_rate))

y_noise = y + n
D_noise = get_stft(y_noise, sr, window_length, overlap, sampling_rate)
D_abs, phase = librosa.magphase(D_noise)

Audio(y_noise, rate=sr)

In [75]:
def IBM(S, N):
    ibm = 1 * (S > N)
    return ibm

In [76]:
M = IBM(S_abs, N_abs)

In [77]:
X = np.multiply(D_abs, M)

In [80]:
X_rec = X * phase

In [81]:
win = scipy.signal.hamming(256,"periodic")
audio_rec = librosa.istft(X_rec,
                  hop_length=round(overlap * window_length),
                  win_length=window_length,        
                  window=win)
Audio(audio_rec, rate = 22050)